# Get links to all cars

In [2]:
import requests
import time
import random
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [4]:
base_urls = []
for i in range(0,2000):
  base_url = 'https://turbo.az/autos?page='+str(i)
  base_urls.append(base_url)

In [6]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'}

In [6]:
#all car links
car_links = []
for url in base_urls:
    response = requests.get(url,headers= headers)
    soup = BeautifulSoup(response.text,'lxml')
    product_list = soup.find_all('a','products-i__link')
    for x in product_list:
        car_links.append('https://turbo.az' +x.get('href'))
    time.sleep(2)

In [81]:
len(set(car_links))

20220

In [28]:
car_links_last = [] 
for x in set(car_links):
    car_links_last.append(x)

In [34]:
with open('car_links_last.txt', 'w', encoding='utf-8') as f:
    for item in car_links_last:
        f.write(item + '\n')

In [5]:
with open('car_links_last.txt', 'r', encoding='utf-8') as f:
    car_links_last = [line.strip() for line in f]

In [6]:
len(car_links_last)

20220

# Get HTMLS of all cars

In [14]:
divs = []
for link in car_links_last:
    response =  requests.get(link, headers=headers)
    html = response.content
    soup = BeautifulSoup(html,'lxml')
    div = soup.select('div[class^="product tz-container"]')
    divs.append(div)
    time.sleep(random.uniform(1, 3))

In [15]:
df = pd.DataFrame(columns=[
    'Şəhər', 'Qiymət', 'Marka', 'Model', 'Buraxılış ili', 'Ban növü', 'Rəng', 'Mühərrik',
    'Yürüş', 'Sürətlər qutusu', 'Ötürücü', 'Yeni', 'Yerlərin sayı', 'Vəziyyəti',
    'Hansı bazar üçün yığılıb', 'Qəzalı', 'Sahiblər'
])

In [16]:
df_list= []
for z in range(len(divs)):
    data_dict = {}
    for x in divs[z]:
        label = x.find_all('label', class_='product-properties__i-name')
        value = x.find_all('span', class_='product-properties__i-value')
        price = x.find('div', class_='product-price')
        for j in range(len(label)):
            name,val = label[j].text,value[j].text
            data_dict[name] = val
            data_dict['Qiymət'] = price.text
        df_list.append(data_dict)
df = pd.DataFrame(df_list,columns = df.columns)


In [18]:
df

,Şəhər,Qiymət,Marka,Model,Buraxılış ili,Ban növü,Rəng,Mühərrik,Yürüş,Sürətlər qutusu,Ötürücü,Yeni,Yerlərin sayı,Vəziyyəti,Hansı bazar üçün yığılıb,Qəzalı,Sahiblər
0,Bərdə,8 500 AZN,GAZ,53,1992,Yük maşını,Yaş Asfalt,4.3 L / 150 a.g. / Benzin,500 000 km,Mexaniki (MT),Arxa,Xeyr,NaN,"Vuruğu yoxdur, rənglənməyib",NaN,NaN,NaN
1,Tovuz,8 500 AZN,Mercedes,C 180,1994,Sedan,Gümüşü,1.8 L / 122 a.g. / Benzin,596 890 km,Avtomat (AT),Arxa,Xeyr,5,"Vuruğu yoxdur, rənglənib",Amerika,NaN,NaN
2,Ağdaş,2 500 AZN,LADA (VAZ),2108,1989,"Hetçbek, 3 qapı",Sarı,1.5 L / 78 a.g. / Benzin,172 820 km,Mexaniki (MT),Ön,Xeyr,5,"Vuruğu var, rənglənib",NaN,NaN,NaN
3,Sumqayıt,28 900 AZN,BMW,528,2012,Sedan,Ağ,2.0 L / 245 a.g. / Benzin,152 410 km,Avtomat (AT),Arxa,Xeyr,5,"Vuruğu yoxdur, rənglənməyib",Amerika,NaN,NaN
4,Bakı,21 500 AZN,Hyundai,ix35,2012,"Offroader / SUV, 5 qapı",Ağ,2.0 L / 150 a.g. / Benzin,270 000 km,Avtomat (AT),Ön,Xeyr,5,"Vuruğu yoxdur, rənglənməyib",Rəsmi diler,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4031,Bakı,14 300 AZN,Mercedes,C 250,2008,Sedan,Ağ,1.8 L / 204 a.g. / Benzin,248 850 km,Avtomat (AT),Arxa,Xeyr,5,"Vuruğu yoxdur, rənglənməyib",Amerika,NaN,NaN
4032,Bakı,19 800 AZN,Ford,Fusion (North America),2017,"Offroader / SUV, 5 qapı",Yaş Asfalt,1.5 L / 184 a.g. / Benzin,142 000 km,Avtomat (AT),Ön,Xeyr,5,"Vuruğu yoxdur, rənglənməyib",Digər,NaN,NaN
4033,Bakı,34 000 AZN,Hyundai,Sonata,2020,Sedan,Qara,1.6 L / 183 a.g. / Benzin,70 000 km,Avtomat (AT),Ön,Xeyr,5,"Vuruğu yoxdur, rənglənib",Amerika,NaN,NaN
4034,Bakı,≈ 86 360 AZN50 800 USD,Hyundai,Grandeur,2023,Sedan,Qara,1.6 L / 230 a.g. / Hibrid,25 000 km,Avtomat (AT),Ön,Xeyr,NaN,"Vuruğu yoxdur, rənglənməyib",NaN,NaN,NaN


In [20]:
df.to_excel('turbo_az_dataset.xlsx',index = False)